In [3]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from datasets import Dataset
from torch.utils.data import DataLoader

In [4]:
import pandas as pd
# 데이터 로드
file_path = "윤석열1.csv"
df = pd.read_csv(file_path)

# 필요한 컬럼만 선택 (결측값 제거)
df = df[['본문', '요약']].dropna()

In [5]:
# 2. 토크나이저 및 모델 불러오기
model_name = "digit82/kobart-summarization"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/295 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/109 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


pytorch_model.bin:   0%|          | 0.00/496M [00:00<?, ?B/s]

In [6]:
# 3. 데이터 토큰화 함수 정의
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["본문"],
        max_length=max_input_length,
        truncation=True,
        padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["요약"],
            max_length=max_target_length,
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [7]:
# 4. Hugging Face Dataset으로 변환 및 토큰화
dataset = Dataset.from_pandas(df)
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

In [8]:
# 5. 데이터셋 분할
train_test_split = tokenized_datasets.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

In [9]:
# 6. 훈련 설정
training_args = TrainingArguments(
    output_dir="./kobart-summarization",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=2,
    save_strategy="epoch",
    num_train_epochs=5,
    load_best_model_at_end=True,
    logging_dir="./logs",
    logging_steps=100
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
# 7. Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

<ipython-input-10-1b93e6c600f3>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [11]:
# 8. 모델 학습 순서ㅇㄹㅇㅇㄹ
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: a01021121569 (a01021121569-aiffel) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,0.421000,0.379672
2,0.311600,0.368531
3,0.225300,0.369864
4,0.176800,0.378881
5,0.147000,0.382306


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=3940, training_loss=0.300530001596751, metrics={'train_runtime': 2677.6548, 'train_samples_per_second': 11.764, 'train_steps_per_second': 1.471, 'total_flos': 9603349217280000.0, 'train_loss': 0.300530001596751, 'epoch': 5.0})

In [12]:
# 9. 모델 저장
trainer.save_model("./kobart-summarization-trained")

In [14]:
# 10. 평가
results = trainer.evaluate()
print(results)

{'eval_loss': 0.3685307204723358, 'eval_runtime': 16.495, 'eval_samples_per_second': 42.437, 'eval_steps_per_second': 5.335, 'epoch': 5.0}


## 모델 테스트

In [33]:
import torch
import re
from collections import Counter
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# CPU에서 실행하도록 설정
device = torch.device("cpu")
# 1. 모델 및 토크나이저 로드
model_path = "./kobart-summarization-trained"  # 학습된 모델 경로
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [34]:
# 요약 생성 함수
def generate_summary(text, max_input_length=256, max_output_length=128):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        max_length=max_input_length,
        truncation=True
    ).to(device)

    with torch.no_grad():
        summary_ids = model.generate(
            inputs["input_ids"],
            max_length=max_output_length,
            num_beams=5,  # 빔 서치 사용
            early_stopping=True
        )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary


In [47]:
# 새로운 기사 입력
new_article = """
홍준표 대구시장은 13일 헌법재판소를 겨냥해 “헌재의 불공정한 심판 과정도 앞으로 개헌 논의 때 존폐 문제가 제기될 것”이라고 밝혔다.

홍 시장은 이날 자신의 페이스북 글에서 “박근혜 탄핵도 위법 결론을 내어 파면시키고 이번에도 똑같은 사태가 일어난다면 개헌 시 헌재를 폐지하고 대법관 정원을 4명 늘려 대법원에 헌법심판부를 두자는 주장도 나올 수 있다”며 이같이 말했다.

그는 비상계엄과 탄핵에 대해선 “부적절한 비상계엄이었지만 그건 대통령의 헌법상 비상대권에 속하기 때문에 불법이라고 볼 수 없고 그걸로 탄핵하는 건 부당하다”고 했다.

이어 “특히 성립이 안 되는 내란죄 프레임을 씌워 불법영장으로 체포 구금하고 구속기간이 만료 되었음에도 불법구속 상태로 기소한 검찰의 만행도 규탄 받을 수밖에 없을 것”이라고 주장했다

그는 또 “우리 당에서도 비상계엄에 찬성한 사람은 없지만 탄핵에 찬성한 반란자들이 일부 있다”면서 “이들은 앞으로 우리당에서 정치하기가 어려울 것”이라고 날을 세웠다. 그러면서 “박근혜 탄핵을 주도한 이들이 퇴출되었듯 앞으로 이들도 정계에서 퇴출 될 수밖에 없을 것”이라고 경고했다.

"""

In [48]:
# 요약 실행
summary = generate_summary(new_article)

In [51]:
print(summary)

대구시장은 13일 헌법재판소를 겨냥해 "헌재의 불공정한 심판 과정도 앞으로 개헌 논의 때 존폐 문제가 제기될 것"이라고 밝혔으며 특히 성립이 안 되는 내란죄 프레임을 씌워 불법구속 상태로 기소한 검찰의 만행도 규탄 받을 수밖에 없을 것이라고 주장하였다.


In [50]:
import shutil
from google.colab import files

# 폴더를 zip 파일로 압축
shutil.make_archive('kobart-summarization-trained', 'zip', 'kobart-summarization-trained')

# 로컬로 다운로드
files.download('kobart-summarization-trained.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>